<a href="https://colab.research.google.com/github/elmomote/DL2024-FacialKeypointDetectionProject/blob/main/%5BF24_DL%5D_Facial_Keypoint_Detection_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Data set from Kaggle

In [4]:
## trying to load data from kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c facial-keypoints-detection
!unzip facial-keypoints-detection.zip
!unzip training.zip
!unzip test.zip


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
unzip:  cannot find or open facial-keypoints-detection.zip, facial-keypoints-detection.zip.zip or facial-keypoints-detection.zip.ZIP.
unzip:  cannot find or open training.zip, training.zip.zip or training.zip.ZIP.
unzip:  cannot find or open test.zip, test.zip.zip or test.zip.ZIP.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

data = pd.read_csv('training.csv')
data['Image'] = data['Image'].apply(lambda x: np.fromstring(x, sep=' '))
image_index = 0  # Change this to display a different image
image = data['Image'][image_index].reshape(96, 96)

plt.imshow(image, cmap='gray')
plt.axis('off')
plt.title("Sample Training Image")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'training.csv'

In [ ]:
keypoints = data.iloc[image_index, :-1].values  # All columns except the 'Image'
keypoints = keypoints.astype(float).reshape(-1, 2)  # Reshape into (x, y) pairs

plt.imshow(image, cmap='gray')
plt.scatter(keypoints[:, 0], keypoints[:, 1], s=20, c='red', marker='o')
plt.axis('off')
plt.title("Training Image with Keypoints")
plt.show()


### **Model # 2**

[Attempt 1]

Create a 80/20 split on the training data. Only use test data when model seems promising.

Creating a simple convolutional model with 8 and filters. Adding option to change if changes needed.

In [ ]:
def m2_simple_convolution(filters = [8, 4]):
  model = tf.keras.models.Sequential()

  model.add(tf.keras.layers.Input(shape = (96, 96, 1)))

  for num_filters in filters:
    model.add(
      tf.keras.layers.Conv2D(
        num_filters,
        kernel_size = (3, 3),
        padding = 'same'
      )
    )

  # Final layer to get the same output shape
  model.add(
    tf.keras.layers.Conv2D(
      1,
      kernel_size = (3, 3),
      padding = 'same'
    )
  )

  model.add(tf.keras.layers.Flatten())

  model.add(tf.keras.layers.Dense(30, activation = 'sigmoid'))

  return model

In [ ]:
m2_attempt_1 = m2_simple_convolution()
m2_attempt_1.summary()